In [1]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px
import dask.dataframe as dd

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

/home/gabriel/anaconda3/envs/.conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/gabriel/.local/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [2]:
caminho_arquivo_csv = '/home/gabriel/dbt-projects/python_files/analises/files/meus-contatos.csv'

dados_csv = pd.read_csv(caminho_arquivo_csv)

dados_csv = dados_csv[dados_csv['CPF'].str.len() == 11]

/tmp/ipykernel_27933/2692619984.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_csv = pd.read_csv(caminho_arquivo_csv)


In [3]:
grouped_df = dados_csv.groupby('Record ID')
dados_csv = grouped_df.first()

In [4]:
dados_csv = dados_csv.reset_index()

In [5]:
# Manter apenas as colunas 'Record ID' e 'CPF' e descartar as outras
dados_csv = dados_csv[['Record ID', 'CPF']]

In [6]:
teste = dados_csv[dados_csv['Record ID'] == 835795321]

In [7]:
import dask.dataframe as dd

dtype_specification = {'CPF': 'object', 'First name': 'object'}

dados_csv = dd.read_csv('/home/gabriel/dbt-projects/python_files/analises/files/meus-contatos.csv', dtype=dtype_specification)

dados_csv['CPF'] = dados_csv['CPF'].astype(str)

# Converta a coluna 'Record ID' para tipo numérico
dados_csv['Record ID'] = dados_csv['Record ID'].astype(int)

# Encontre o máximo do 'Record ID' para cada grupo 'CPF'
max_record_id = dados_csv.groupby('CPF')['Record ID'].max().reset_index()

# Converta o Dask DataFrame em um pandas DataFrame, se necessário
dados_csv = max_record_id.compute()

: 

In [ ]:
#verificar se de fato, está trazendo o máximo do Record ID
validacao = dados_csv[dados_csv['CPF'] == '81945515520']

In [ ]:
ddf2 = dd.read_csv('/home/gabriel/dbt-projects/python_files/analises/files/fl_consultas_crm_202401041558 (4).csv', dtype={'CPF': 'object'})
ddf2['CPF'] = ddf2['CPF'].astype('str')
ddf2 = ddf2.compute()

In [ ]:
dados_csv = dados_csv.set_index('CPF')
ddf2 = ddf2.set_index('CPF')
ddf_merged = dd.merge(ddf2, dados_csv, left_index=True, right_index=True, how='inner')

: 

In [ ]:
# Salvar o resultado como CSV
ddf_merged.to_csv('resultado.csv', single_file=True, index=False)

: 